In [8]:
import json
import requests
import base64
from itertools import islice
import matplotlib as mp
from bs4 import BeautifulSoup
import re
from typing import Dict, List, Any 
import pandas as pd
from requests.auth import HTTPBasicAuth

In [ ]:
#Assigning a value to each one of these variables 

# Can be found in the link of the query when in ADev Ops
organization = '{organization}'

# Can be found in the link of the query fro bridgestone - its the Porfolio you are trying to access
project = '{project_name}'

#query id can be found at the end of the link when you are viewing the query
query_id = '{query_id}'

#generated through account setting in the top right its a person with a gear then you click "personal Access tokens"
pat_token = "{pat_token}"

In [10]:
        # https://dev.azure.com/ - base url for dev ops REST API indicating that we are accessing Azure DevOps services.
        # Organization - is Bridgestoneglobal which you assigned above.
        # Project-  is the portfolio you are trying to access which is assigned above.
        # _apis/wit/wiql/ - This indicates that the API being accessed is under the Work Item Tracking (WIT) service (part of the Azure DevOps Services REST API),specifically the WIQL (Work Item Query Language) endpoint. WIQL is used for querying work items.
        # Query ID - indicated which query you are trying to access it's assigned above
        #Apiv version - specifies which version of the API you are using.  you can find this in the documentation at this link. https://learn.microsoft.com/en-us/rest/api/azure/devops/wit/work-items?view=azure-devops-rest-7.1
        
url = f'https://dev.azure.com/{organization}/{project}/_apis/wit/wiql/{query_id}?api-version=7.1'


To sum up, developing URLs for API calls is essential for:

Precisely targeting and interacting with specific resources.
Ensuring structured, standardized, and flexible access.
Enabling integration and automation across different systems and applications.
Understanding and constructing these URLs accurately is crucial for leveraging the full functionality of RESTful APIs like those provided by Azure DevOps.

In [11]:
#This line is making an HTTP GET request to the specified URL using the requests library in Python.
#requests.get -This is a method from the requests library that is used to send an HTTP GET request. The GET method is used to retrieve data from the server specified by the URL.
# url - This is the URL to which the GET request is sent. It was created above
# auth = HTTPBasicAuth - This is a class provided by the requests library in the requests.auth module. It is used to set up Basic Authentication for the request. The empty string in this context is for the username which we dont need becasue we are using a personal access token. 
# pat token - the token created in Azure Dev ops. 


response = requests.get(url, auth=HTTPBasicAuth('', pat_token))

In [12]:
requests

<module 'requests' from 'c:\\Users\\cw110036\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\requests\\__init__.py'>

In [13]:
# setting up error troubleshooting to give you an error code to troubleshoot rather than it throwing a random error.

if response.status_code == 200:   #if status code is = 200 (successful request)  
    query_results = response.json()   # then provide the query results from 'response' variable above (we are setting it to a new variable here)
    print(query_results)             # print the reponse in a json format
else:
    print(f"Failed to fetch query results. Status code: {response.status_code}") # if you get an error print the orange text with the status code from the response I assume status_code is a function in the request librbary. 
    print(response.text)    #printing the response code.    https://www.restapitutorial.com/httpstatuscodes this link will take you to a list of response codes.

{'queryType': 'flat', 'queryResultType': 'workItem', 'asOf': '2025-07-25T14:13:00.91Z', 'columns': [{'referenceName': 'System.Id', 'name': 'ID', 'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.Id'}, {'referenceName': 'System.WorkItemType', 'name': 'Work Item Type', 'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.WorkItemType'}, {'referenceName': 'System.Title', 'name': 'Title', 'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.Title'}, {'referenceName': 'System.AssignedTo', 'name': 'Assigned To', 'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.AssignedTo'}, {'referenceName': 'System.State', 'name': 'State', 'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.State'}, {'referenceName': 'System.Tags', 'name': 'Tags', 'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.Tags'}, {'referenceName': 'System.TeamProject', 'name': 'Team Project', 'url': 'https://d

In [14]:
pd.json_normalize(query_results)
query_results['columns']      #"Use this to look at what all the column values are"

[{'referenceName': 'System.Id',
  'name': 'ID',
  'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.Id'},
 {'referenceName': 'System.WorkItemType',
  'name': 'Work Item Type',
  'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.WorkItemType'},
 {'referenceName': 'System.Title',
  'name': 'Title',
  'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.Title'},
 {'referenceName': 'System.AssignedTo',
  'name': 'Assigned To',
  'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.AssignedTo'},
 {'referenceName': 'System.State',
  'name': 'State',
  'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.State'},
 {'referenceName': 'System.Tags',
  'name': 'Tags',
  'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.Tags'},
 {'referenceName': 'System.TeamProject',
  'name': 'Team Project',
  'url': 'https://dev.azure.com/BridgestoneGlobal/_apis/wit/fields/System.TeamProjec

In [15]:
# Overview of the code: The function extract_work_item_ids is designed to safely extract and return a list of work item IDs from the query_results dictionary.

def extract_work_item_ids(query_results): # you are defining the the function 'extract_work_item_ids' and it take one parameter
    
    work_item_ids =[]                      # An empty list work_item_ids is initialized. This list will be used to store the work item IDs extracted from query_results.
    
    work_items = query_results.get('workItems',[])  # The code attempts to get the value associated with the key 'workItems' from query_results. If no item exist under 'workItems' then it will default to an empty list.
    
    for work_item in work_items:               # for loop created to iterate over each work_item in the work_items list. So for each 'work_item' (you are creating this variable now) in 'work_items' (you assigned this variable earlier which is getting the query results for work Items in the JSON format) a work itme would be one of the blocks in the curly brakets {}  {'id': 131743, 'url': 'https://dev.azure.com/BridgestoneGlobal/44e6f094-1ced-43e6-aebd-92eb2a801c96/_apis/wit/workItems/131743'}, {'id': 131753, 'url': 'https://dev.azure.com/BridgestoneGlobal/44e6f094-1ced-43e6-aebd-92eb2a801c96/_apis/wit/workItems/131753'},
        work_item_id = work_item.get('id')      # for each work item you are attempting to extract each work item id through this fucntion. 
        if work_item_id is not None:              # this is a check to ensure there is a 'id' in a work_item
            work_item_ids.append(work_item_id)     # If work_item_id is valid (i.e., not None), it gets appended to the work_item_ids list.
            
    return work_item_ids  # After the loop ends, the function returns the work_item_ids list, which now contains all the work item IDs extracted from the query_results.
         

In [16]:
work_item_ids = extract_work_item_ids(query_results) # The function will retrieve the list of work items under the 'workItems' key. It will iterate over this list, extract the id from each work item, and append the IDs to work_item_ids.
print(work_item_ids) #Printing the work item list.

[373586, 373588, 441625, 475391, 496004, 515471, 519506, 531948, 532022, 534249, 534251, 534252, 535733, 535742, 535760, 535763, 535767, 537128, 537130, 537131, 537132, 537133, 537134, 537438, 540294, 540441, 540493, 540605, 540611, 540612, 540614, 540615, 540618, 540744, 541077, 541896, 542064, 542169, 542332, 542342, 542343, 542344, 542348, 542351, 542353, 543304, 543326, 544758, 545211, 545223, 545376, 545377, 545684, 545686, 545688, 545689, 545690, 546385, 546386, 548176, 549312, 549317, 552105, 552106, 552107, 552108, 552109, 552110, 552111, 552112, 552113, 553992, 553993, 553996, 554915, 556304, 558740, 558911, 559529, 559534, 560678, 560690, 560694, 561343, 561372, 561436, 561489, 566780, 571305, 571326, 572709, 573302, 574666, 574824, 575991, 583108, 583794, 583797, 583805, 583813, 585519, 585529, 585536, 585540, 585547, 585885, 586052, 586054, 586065, 586067, 586070, 586073, 586075, 586081, 586082, 586561]


BREAK BETWEEN 2 DIFFERENT API CALLS

In [17]:
#define all work item ID's to a variable. THE IDS IN HERE CAN CREATE AN ERROR so I had to set 

batch_work_item_ids = work_item_ids
batch_work_item_ids

[373586,
 373588,
 441625,
 475391,
 496004,
 515471,
 519506,
 531948,
 532022,
 534249,
 534251,
 534252,
 535733,
 535742,
 535760,
 535763,
 535767,
 537128,
 537130,
 537131,
 537132,
 537133,
 537134,
 537438,
 540294,
 540441,
 540493,
 540605,
 540611,
 540612,
 540614,
 540615,
 540618,
 540744,
 541077,
 541896,
 542064,
 542169,
 542332,
 542342,
 542343,
 542344,
 542348,
 542351,
 542353,
 543304,
 543326,
 544758,
 545211,
 545223,
 545376,
 545377,
 545684,
 545686,
 545688,
 545689,
 545690,
 546385,
 546386,
 548176,
 549312,
 549317,
 552105,
 552106,
 552107,
 552108,
 552109,
 552110,
 552111,
 552112,
 552113,
 553992,
 553993,
 553996,
 554915,
 556304,
 558740,
 558911,
 559529,
 559534,
 560678,
 560690,
 560694,
 561343,
 561372,
 561436,
 561489,
 566780,
 571305,
 571326,
 572709,
 573302,
 574666,
 574824,
 575991,
 583108,
 583794,
 583797,
 583805,
 583813,
 585519,
 585529,
 585536,
 585540,
 585547,
 585885,
 586052,
 586054,
 586065,
 586067,
 586070,
 

In [ ]:
def batch_request(all_work_item_ids, batch_size=200): #The function batch_request is defined to take two parameters - all_work_item_ids: A list of work item IDs that you want to fetch details for. batch_size: The number of work item IDs to include in each batch request (default is 200).

    batch_url = 'https://dev.azure.com/{OrganizationName}/_apis/wit/workItemsBatch?api-version=7.1' # The URL endpoint for batch requests to the Azure DevOps API.
    batch_pat_token = '{batach_token}' # token created in AZ dev ops
    encoded_pat = base64.b64encode(bytes(':'+ batch_pat_token, 'utf-8')).decode('utf-8') #: The PAT is encoded in Base64 format for inclusion in the HTTP Basic Auth header. NEED MORE HERE ON WHY
    headers = {                                 #headers is a dictionary that sets the Content-Type to 'application/json' and the Authorization to use the encoded PAT.
        'Content-Type': 'application/json',
        'Authorization': 'Basic ' + encoded_pat
    }
     #helper function to split data into batches (the api will only let you do 200 at a tiime)
    def chunks(data, size):   #The chunks function splits a list (data) into smaller lists (batches) of a given size.  '' How does it know what data an size are '' 
        it = iter(data)       # Creates an iterator for the data.
        for i in range(0, len(data), size): ### dont forget the I IS NOT BEING ACCESSED 
            yield list(islice(it, size))   #The for loop iterates over data in increments of size, yielding batches as lists.

    batch_work_item_details = [] # initializing or making a list to store the details of the work items.

    for batch in chunks(all_work_item_ids, batch_size):  #The for loop iterates over the batched work item IDs created by the chunks function.
        body = {                                                                                      #A dictionary body is created containing:
            "ids": batch,                                                                              #"ids": batch: The current batch of work item IDs.  
            "fields": [                                                                                 # A list of fields to retrieve for each work item.
                "System.Id", "System.WorkItemType", "System.Title", "System.AssignedTo", "System.State", "System.Tags","System.TeamProject" , "Microsoft.VSTS.Scheduling.TargetDate",
                'Custom.EstimatedStartDate',"System.CreatedDate","Custom.WSJF",'Microsoft.VSTS.Scheduling.Effort','Microsoft.VSTS.Common.BusinessValue', "System.Description", "Custom.BusinessOutcomes", 
                 "Custom.LeadingIndicators", "System.Description", "Custom.LeadingIndicators" , "Custom.NonFunctionalRequirements", "Custom.Threshold",'Custom.ValueDriver'
                 ]
        }
        # Making the request to Azure DevOps - Making the HTTP POST Request
        response2 = requests.post(batch_url, headers=headers, data=json.dumps(body)) #Sending the batched request. json.dumps converts the body dictionary to a string. 
        
        #Handling the Response:
        if response2.status_code == 200:
            batch_work_item_details.extend(response2.json()['value']) #If the request is successful (status_code == 200), the work item details retrieved in the response are extended into batch_work_item_details ( The empty list created above).
        else:
            response2.raise_for_status() # Raises a response if there is an error

    return batch_work_item_details   #The function returns batch_work_item_details, which contains the combined details of all the work items retrieved from the batch requests.

In [19]:
#NOTE: ALL REFERENCE NAMES NEED TO BE ACCURATE WITH THE CORRRECT PREFIX FOR AN ERROR TO NOT BE THROW.
result = batch_request(batch_work_item_ids) # Calling the "batch_request" fucntion 
html = json.dumps(result, indent =2) # this converts the result variable (a python list) into JSON string which is shown below
print(html)

[
  {
    "id": 373586,
    "rev": 83,
    "fields": {
      "System.Id": 373586,
      "System.TeamProject": "Modern IT Management",
      "System.WorkItemType": "Portfolio Epic",
      "System.State": "Implement",
      "System.AssignedTo": {
        "displayName": "Gade, Brent",
        "url": "https://spsprodcus2.vssps.visualstudio.com/A7d8e14d1-cc00-41d1-80fd-40606e261b45/_apis/Identities/33ebd45c-18ac-6760-8ed7-8e5f985582eb",
        "_links": {
          "avatar": {
            "href": "https://dev.azure.com/BridgestoneGlobal/_apis/GraphProfile/MemberAvatars/aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NTU4MmVi"
          }
        },
        "id": "33ebd45c-18ac-6760-8ed7-8e5f985582eb",
        "uniqueName": "GadeBrent@bfusa.com",
        "imageUrl": "https://dev.azure.com/BridgestoneGlobal/_apis/GraphProfile/MemberAvatars/aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NTU4MmVi",
        "descriptor": "aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NTU4MmVi"
      },
      "System.Cre

In [20]:
def remove_html_tags(text: str) -> str:     ## this function removes HTML tags from a string. The parameters "text" a string that contains html tags. and returns "clean text" text with the html characters removed
    if not text:   #If the text is "none" or empty return an empty string 
        return ""  # return empty string
    soup = BeautifulSoup(text, "html.parser")  #BeautifulSoup parses the HTML and soup.get_text(separator=" ") retrieves the text without tags, with words separated by spaces.
    clean_text = soup.get_text(separator=" ")
    return clean_text

def replace_unicode_chars(text: str) -> str: #Purpose: Replaces specific Unicode characters in a string with more common characters.
    replacements = {                           #A dictionary replacements maps Unicode characters to their replacements.
        '\u2013': '-',    # En dash
        '\u200b': '',    # Zero-width space
        '\u201c': '"',   # Left double quotation mark
        '\u201d': '"',   # Right double quotation mark
    }
    for old, new in replacements.items():     #Loop through the dictionary and replace occurrences of each Unicode character in the string with its corresponding replacement. How does it knwo odl and new?
        text = text.replace(old, new)           #The .replace() function in Python is a built-in string method used to replace occurrences of a specified substring with another substring. string.replace(old, new, count)
    return text

def remove_extra_spaces(text: str) -> str:     #Purpose: Remove extra spaces from a string and trim leading/trailing spaces.
    return re.sub(r'\s+', ' ', text).strip()        #re.sub(r'\s+', ' ', text) replaces multiple spaces with a single space. strip() removes leading and trailing spaces.

def clean_text_in_keys(data: Dict[str, Any], keys_to_clean: List[str]) -> Dict[str, Any]:   #Purpose: Cleans text in specified keys of a dictionary, including nested dictionaries and lists.
    for k, v in data.items():                                                                  #data (Dict[str, Any]): The dictionary containing data to clean. K is considered the key in the dictionary v is considered the value or the multi line text 
        if isinstance(v, str) and k in keys_to_clean:    #Checks if value is of type string.     #If the key k is found in keys_to_clean and the corresponding value v is a string, the value v needs to be cleaned
            # Clean string values that match specified keys
            v = remove_html_tags(v)    #Cleaning values of k (Keys)
            v = replace_unicode_chars(v)     #Cleaning values of k (Keys)
            v = remove_extra_spaces(v)        #Cleaning values of k (Keys)
            data[k] = v                             #sets the cleaned value v back to the key k in the dictionary.
        elif isinstance(v, dict):                 #Checks if value is of type dictionary if it is clean the values.     The isinstance() function in Python is used to check if an object is an instance or subclass of a specified class or tuple of classes. It returns True if the object is an instance or subclass of the specified class, and False otherwise. This function is commonly used to perform type checking in Python programs.
            # Recursively clean dictionary values
            data[k] = clean_text_in_keys(v, keys_to_clean)
        elif isinstance(v, list):
            # Clean items in the list if they are dictionaries
            data[k] = [clean_text_in_keys(item, keys_to_clean) if isinstance(item, dict) else item for item in v]
    return data

html2 = json.loads(html)

keys_to_clean  = [ "Custom.LeadingIndicators","Custom.BusinessOutcomes","System.Description","Custom.NonFunctionalRequirements"]

cleaned_data = [clean_text_in_keys(item, keys_to_clean) for item in html2]
print(json.dumps(cleaned_data, indent=2))

[
  {
    "id": 373586,
    "rev": 83,
    "fields": {
      "System.Id": 373586,
      "System.TeamProject": "Modern IT Management",
      "System.WorkItemType": "Portfolio Epic",
      "System.State": "Implement",
      "System.AssignedTo": {
        "displayName": "Gade, Brent",
        "url": "https://spsprodcus2.vssps.visualstudio.com/A7d8e14d1-cc00-41d1-80fd-40606e261b45/_apis/Identities/33ebd45c-18ac-6760-8ed7-8e5f985582eb",
        "_links": {
          "avatar": {
            "href": "https://dev.azure.com/BridgestoneGlobal/_apis/GraphProfile/MemberAvatars/aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NTU4MmVi"
          }
        },
        "id": "33ebd45c-18ac-6760-8ed7-8e5f985582eb",
        "uniqueName": "GadeBrent@bfusa.com",
        "imageUrl": "https://dev.azure.com/BridgestoneGlobal/_apis/GraphProfile/MemberAvatars/aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NTU4MmVi",
        "descriptor": "aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NTU4MmVi"
      },
      "System.Cre

In [21]:
df = pd.json_normalize(cleaned_data)
az_df = pd.DataFrame(df)

# Renaming column names
az_df.columns = az_df.columns.str.replace('^fields.', '', regex=True)
az_df.columns = az_df.columns.str.replace('^System.', '', regex=True)
az_df.columns = az_df.columns.str.replace('^Microsoft.VSTS.Scheduling.', '', regex=True)
az_df.columns = az_df.columns.str.replace('^Custom', '', regex=True)
az_df.columns = az_df.columns.str.replace('^AssignedTo', '', regex=True)
az_df.columns = az_df.columns.str.replace('^\.', '', regex=True)
az_df.columns.to_list()

<>:10: SyntaxWarning: invalid escape sequence '\.'
<>:10: SyntaxWarning: invalid escape sequence '\.'
C:\Users\cw110036\AppData\Local\Temp\ipykernel_25188\2587961754.py:10: SyntaxWarning: invalid escape sequence '\.'
  az_df.columns = az_df.columns.str.replace('^\.', '', regex=True)


['id',
 'rev',
 'url',
 'Id',
 'TeamProject',
 'WorkItemType',
 'State',
 'displayName',
 'url',
 '_links.avatar.href',
 'id',
 'uniqueName',
 'imageUrl',
 'descriptor',
 'CreatedDate',
 'Title',
 'TargetDate',
 'Microsoft.VSTS.Common.BusinessValue',
 'Effort',
 'WSJF',
 'Description',
 'BusinessOutcomes',
 'LeadingIndicators',
 'ValueDriver',
 'Tags',
 'EstimatedStartDate',
 'NonFunctionalRequirements',
 'commentVersionRef.commentId',
 'commentVersionRef.version',
 'commentVersionRef.url']

In [22]:
az_df = az_df.fillna('')



In [23]:
# Drop the column at index position 2



In [24]:
IT_BigRocks_df= az_df[[
'id',
 #'rev',
 #'url',
 'Id',
 'TeamProject',
 'WorkItemType',
 'State',
 'displayName',
 #'url',
 '_links.avatar.href',
 #'id',
 'uniqueName',
 'imageUrl',
 'descriptor',
 'CreatedDate',
 'Title',
 'TargetDate',
 'Effort',
 'Microsoft.VSTS.Common.BusinessValue',
 'WSJF',
 'Description',
 'BusinessOutcomes',
 'LeadingIndicators',
 'NonFunctionalRequirements',
 'Tags',
 'EstimatedStartDate',
 'ValueDriver'
 #'commentVersionRef.commentId',
 #'commentVersionRef.version',
 #'commentVersionRef.url'
 
 ]]
IT_BigRocks_df

,id,id,Id,TeamProject,WorkItemType,State,displayName,_links.avatar.href,uniqueName,imageUrl,...,Effort,Microsoft.VSTS.Common.BusinessValue,WSJF,Description,BusinessOutcomes,LeadingIndicators,NonFunctionalRequirements,Tags,EstimatedStartDate,ValueDriver
0,373586,33ebd45c-18ac-6760-8ed7-8e5f985582eb,373586,Modern IT Management,Portfolio Epic,Implement,"Gade, Brent",https://dev.azure.com/BridgestoneGlobal/_apis/...,GadeBrent@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,...,8.4,7.0,0.833333,Migrate and modernize BSAM applications and da...,"Create a secure, scalable cloud platform Accel...",Mainframe migration Q4 2024 Migration Waves 13...,,,,
1,373588,33ebd45c-18ac-6760-8ed7-8e5f985582eb,373588,Modern IT Management,Portfolio Epic,Implement,"Gade, Brent",https://dev.azure.com/BridgestoneGlobal/_apis/...,GadeBrent@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,...,7.45,7.0,0.939597,Reduce capital and operating cost of applicati...,,,,,,
2,441625,b33d4c12-640e-668f-beb8-218f94a07c6e,441625,Manufacture Quality Products,Portfolio Epic,Plan,"Harden, Wendy",https://dev.azure.com/BridgestoneGlobal/_apis/...,HardenWendy@BFUSA.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,...,2.8,10.0,3.571429,Ministry of Finance in Poland is requiring tha...,Avoid non-compliance penalties. Per FSA Legal:...,,,Big Rock,,Blue Dolars
3,475391,b33d4c12-640e-668f-beb8-218f94a07c6e,475391,Manufacture Quality Products,Portfolio Epic,Plan,"Harden, Wendy",https://dev.azure.com/BridgestoneGlobal/_apis/...,HardenWendy@BFUSA.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,...,2.8,10.0,3.571429,Ministry of Finance in Poland is requiring bus...,e-Invoicing - Risk Mitigation - non-compliance...,,,Big Rock,,Blue Dolars
4,496004,87df3169-a2fe-6e36-a050-52f543c7d02f,496004,Customer and Brand,Portfolio Epic,Implement,"Boyle, Paola Erin",https://dev.azure.com/BridgestoneGlobal/_apis/...,BoyleErin1@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,...,6.5,4.0,3.076923,Support Marketing teams implementing B2C C360 ...,Related 2025 IT Goals: Fundamentally improve o...,,,2025.Q2; 2025.Q3; 2025.Q4; Enterprise & Custom...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,586073,4869cf5f-f37d-65ac-a3df-8439e315de2f,586073,Secure and Protect the Enterprise,Portfolio Epic,Implement,Todd Thiel,https://dev.azure.com/BridgestoneGlobal/_apis/...,ThielTodd@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,...,6.45,8.0,1.240310,,,,,,,
112,586075,4869cf5f-f37d-65ac-a3df-8439e315de2f,586075,Secure and Protect the Enterprise,Portfolio Epic,Implement,Todd Thiel,https://dev.azure.com/BridgestoneGlobal/_apis/...,ThielTodd@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,...,6.45,8.0,1.240310,,,,,,,
113,586081,23c08155-ecfa-6c2d-b8d4-ce5fe990c8e6,586081,Secure and Protect the Enterprise,Portfolio Epic,Implement,Jeremy Pade,https://dev.azure.com/BridgestoneGlobal/_apis/...,PadeJeremy@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,...,7.35,9.0,1.224490,,,,,,,
114,586082,23c08155-ecfa-6c2d-b8d4-ce5fe990c8e6,586082,Secure and Protect the Enterprise,Portfolio Epic,Implement,Jeremy Pade,https://dev.azure.com/BridgestoneGlobal/_apis/...,PadeJeremy@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,...,7.35,9.0,1.224490,,,,,,,


In [25]:
IT_BigRocks_df = IT_BigRocks_df.drop(IT_BigRocks_df.columns[1], axis=1)

In [26]:
IT_BigRocks_df

,Id,TeamProject,WorkItemType,State,displayName,_links.avatar.href,uniqueName,imageUrl,descriptor,CreatedDate,...,Effort,Microsoft.VSTS.Common.BusinessValue,WSJF,Description,BusinessOutcomes,LeadingIndicators,NonFunctionalRequirements,Tags,EstimatedStartDate,ValueDriver
0,373586,Modern IT Management,Portfolio Epic,Implement,"Gade, Brent",https://dev.azure.com/BridgestoneGlobal/_apis/...,GadeBrent@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NT...,2023-08-07T15:53:28.51Z,...,8.4,7.0,0.833333,Migrate and modernize BSAM applications and da...,"Create a secure, scalable cloud platform Accel...",Mainframe migration Q4 2024 Migration Waves 13...,,,,
1,373588,Modern IT Management,Portfolio Epic,Implement,"Gade, Brent",https://dev.azure.com/BridgestoneGlobal/_apis/...,GadeBrent@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NT...,2023-08-07T15:53:47.827Z,...,7.45,7.0,0.939597,Reduce capital and operating cost of applicati...,,,,,,
2,441625,Manufacture Quality Products,Portfolio Epic,Plan,"Harden, Wendy",https://dev.azure.com/BridgestoneGlobal/_apis/...,HardenWendy@BFUSA.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.YjMzZDRjMTItNjQwZS03NjhmLWJlYjgtMjE4Zjk0YT...,2024-03-29T06:26:35.01Z,...,2.8,10.0,3.571429,Ministry of Finance in Poland is requiring tha...,Avoid non-compliance penalties. Per FSA Legal:...,,,Big Rock,,Blue Dolars
3,475391,Manufacture Quality Products,Portfolio Epic,Plan,"Harden, Wendy",https://dev.azure.com/BridgestoneGlobal/_apis/...,HardenWendy@BFUSA.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.YjMzZDRjMTItNjQwZS03NjhmLWJlYjgtMjE4Zjk0YT...,2024-07-12T17:53:48.32Z,...,2.8,10.0,3.571429,Ministry of Finance in Poland is requiring bus...,e-Invoicing - Risk Mitigation - non-compliance...,,,Big Rock,,Blue Dolars
4,496004,Customer and Brand,Portfolio Epic,Implement,"Boyle, Paola Erin",https://dev.azure.com/BridgestoneGlobal/_apis/...,BoyleErin1@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NTA4ODUxNzAtZjg5Yy03ZWU2LTljYTYtOWJjNzgyMz...,2024-09-24T18:37:22.933Z,...,6.5,4.0,3.076923,Support Marketing teams implementing B2C C360 ...,Related 2025 IT Goals: Fundamentally improve o...,,,2025.Q2; 2025.Q3; 2025.Q4; Enterprise & Custom...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,586073,Secure and Protect the Enterprise,Portfolio Epic,Implement,Todd Thiel,https://dev.azure.com/BridgestoneGlobal/_apis/...,ThielTodd@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NDg2OWNmNWYtZjM3ZC03NWFjLWEzZGYtODQzOWUzMT...,2025-07-22T19:51:17.163Z,...,6.45,8.0,1.240310,,,,,,,
112,586075,Secure and Protect the Enterprise,Portfolio Epic,Implement,Todd Thiel,https://dev.azure.com/BridgestoneGlobal/_apis/...,ThielTodd@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NDg2OWNmNWYtZjM3ZC03NWFjLWEzZGYtODQzOWUzMT...,2025-07-22T19:52:35.177Z,...,6.45,8.0,1.240310,,,,,,,
113,586081,Secure and Protect the Enterprise,Portfolio Epic,Implement,Jeremy Pade,https://dev.azure.com/BridgestoneGlobal/_apis/...,PadeJeremy@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NTQ4ZGRmMjAtYmM2MC03YzJkLTlhMDktNDY3NGFmOD...,2025-07-22T20:17:16.31Z,...,7.35,9.0,1.224490,,,,,,,
114,586082,Secure and Protect the Enterprise,Portfolio Epic,Implement,Jeremy Pade,https://dev.azure.com/BridgestoneGlobal/_apis/...,PadeJeremy@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NTQ4ZGRmMjAtYmM2MC03YzJkLTlhMDktNDY3NGFmOD...,2025-07-22T20:23:04.257Z,...,7.35,9.0,1.224490,,,,,,,


In [27]:
IT_BigRocks_df.insert(len(IT_BigRocks_df.columns), "Organization", "BridgestoneGlobal/")
IT_BigRocks_df.insert(len(IT_BigRocks_df.columns), "AzureDevRootUrl", "https://dev.azure.com/")


In [28]:
IT_BigRocks_df

,Id,TeamProject,WorkItemType,State,displayName,_links.avatar.href,uniqueName,imageUrl,descriptor,CreatedDate,...,WSJF,Description,BusinessOutcomes,LeadingIndicators,NonFunctionalRequirements,Tags,EstimatedStartDate,ValueDriver,Organization,AzureDevRootUrl
0,373586,Modern IT Management,Portfolio Epic,Implement,"Gade, Brent",https://dev.azure.com/BridgestoneGlobal/_apis/...,GadeBrent@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NT...,2023-08-07T15:53:28.51Z,...,0.833333,Migrate and modernize BSAM applications and da...,"Create a secure, scalable cloud platform Accel...",Mainframe migration Q4 2024 Migration Waves 13...,,,,,BridgestoneGlobal/,https://dev.azure.com/
1,373588,Modern IT Management,Portfolio Epic,Implement,"Gade, Brent",https://dev.azure.com/BridgestoneGlobal/_apis/...,GadeBrent@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NT...,2023-08-07T15:53:47.827Z,...,0.939597,Reduce capital and operating cost of applicati...,,,,,,,BridgestoneGlobal/,https://dev.azure.com/
2,441625,Manufacture Quality Products,Portfolio Epic,Plan,"Harden, Wendy",https://dev.azure.com/BridgestoneGlobal/_apis/...,HardenWendy@BFUSA.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.YjMzZDRjMTItNjQwZS03NjhmLWJlYjgtMjE4Zjk0YT...,2024-03-29T06:26:35.01Z,...,3.571429,Ministry of Finance in Poland is requiring tha...,Avoid non-compliance penalties. Per FSA Legal:...,,,Big Rock,,Blue Dolars,BridgestoneGlobal/,https://dev.azure.com/
3,475391,Manufacture Quality Products,Portfolio Epic,Plan,"Harden, Wendy",https://dev.azure.com/BridgestoneGlobal/_apis/...,HardenWendy@BFUSA.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.YjMzZDRjMTItNjQwZS03NjhmLWJlYjgtMjE4Zjk0YT...,2024-07-12T17:53:48.32Z,...,3.571429,Ministry of Finance in Poland is requiring bus...,e-Invoicing - Risk Mitigation - non-compliance...,,,Big Rock,,Blue Dolars,BridgestoneGlobal/,https://dev.azure.com/
4,496004,Customer and Brand,Portfolio Epic,Implement,"Boyle, Paola Erin",https://dev.azure.com/BridgestoneGlobal/_apis/...,BoyleErin1@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NTA4ODUxNzAtZjg5Yy03ZWU2LTljYTYtOWJjNzgyMz...,2024-09-24T18:37:22.933Z,...,3.076923,Support Marketing teams implementing B2C C360 ...,Related 2025 IT Goals: Fundamentally improve o...,,,2025.Q2; 2025.Q3; 2025.Q4; Enterprise & Custom...,,,BridgestoneGlobal/,https://dev.azure.com/
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,586073,Secure and Protect the Enterprise,Portfolio Epic,Implement,Todd Thiel,https://dev.azure.com/BridgestoneGlobal/_apis/...,ThielTodd@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NDg2OWNmNWYtZjM3ZC03NWFjLWEzZGYtODQzOWUzMT...,2025-07-22T19:51:17.163Z,...,1.240310,,,,,,,,BridgestoneGlobal/,https://dev.azure.com/
112,586075,Secure and Protect the Enterprise,Portfolio Epic,Implement,Todd Thiel,https://dev.azure.com/BridgestoneGlobal/_apis/...,ThielTodd@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NDg2OWNmNWYtZjM3ZC03NWFjLWEzZGYtODQzOWUzMT...,2025-07-22T19:52:35.177Z,...,1.240310,,,,,,,,BridgestoneGlobal/,https://dev.azure.com/
113,586081,Secure and Protect the Enterprise,Portfolio Epic,Implement,Jeremy Pade,https://dev.azure.com/BridgestoneGlobal/_apis/...,PadeJeremy@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NTQ4ZGRmMjAtYmM2MC03YzJkLTlhMDktNDY3NGFmOD...,2025-07-22T20:17:16.31Z,...,1.224490,,,,,,,,BridgestoneGlobal/,https://dev.azure.com/
114,586082,Secure and Protect the Enterprise,Portfolio Epic,Implement,Jeremy Pade,https://dev.azure.com/BridgestoneGlobal/_apis/...,PadeJeremy@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NTQ4ZGRmMjAtYmM2MC03YzJkLTlhMDktNDY3NGFmOD...,2025-07-22T20:23:04.257Z,...,1.224490,,,,,,,,BridgestoneGlobal/,https://dev.azure.com/


In [29]:
IT_BigRocks_df.insert(len(IT_BigRocks_df.columns), "TeamProjectUrl",IT_BigRocks_df["TeamProject"])

In [30]:
IT_BigRocks_df

,Id,TeamProject,WorkItemType,State,displayName,_links.avatar.href,uniqueName,imageUrl,descriptor,CreatedDate,...,Description,BusinessOutcomes,LeadingIndicators,NonFunctionalRequirements,Tags,EstimatedStartDate,ValueDriver,Organization,AzureDevRootUrl,TeamProjectUrl
0,373586,Modern IT Management,Portfolio Epic,Implement,"Gade, Brent",https://dev.azure.com/BridgestoneGlobal/_apis/...,GadeBrent@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NT...,2023-08-07T15:53:28.51Z,...,Migrate and modernize BSAM applications and da...,"Create a secure, scalable cloud platform Accel...",Mainframe migration Q4 2024 Migration Waves 13...,,,,,BridgestoneGlobal/,https://dev.azure.com/,Modern IT Management
1,373588,Modern IT Management,Portfolio Epic,Implement,"Gade, Brent",https://dev.azure.com/BridgestoneGlobal/_apis/...,GadeBrent@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NT...,2023-08-07T15:53:47.827Z,...,Reduce capital and operating cost of applicati...,,,,,,,BridgestoneGlobal/,https://dev.azure.com/,Modern IT Management
2,441625,Manufacture Quality Products,Portfolio Epic,Plan,"Harden, Wendy",https://dev.azure.com/BridgestoneGlobal/_apis/...,HardenWendy@BFUSA.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.YjMzZDRjMTItNjQwZS03NjhmLWJlYjgtMjE4Zjk0YT...,2024-03-29T06:26:35.01Z,...,Ministry of Finance in Poland is requiring tha...,Avoid non-compliance penalties. Per FSA Legal:...,,,Big Rock,,Blue Dolars,BridgestoneGlobal/,https://dev.azure.com/,Manufacture Quality Products
3,475391,Manufacture Quality Products,Portfolio Epic,Plan,"Harden, Wendy",https://dev.azure.com/BridgestoneGlobal/_apis/...,HardenWendy@BFUSA.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.YjMzZDRjMTItNjQwZS03NjhmLWJlYjgtMjE4Zjk0YT...,2024-07-12T17:53:48.32Z,...,Ministry of Finance in Poland is requiring bus...,e-Invoicing - Risk Mitigation - non-compliance...,,,Big Rock,,Blue Dolars,BridgestoneGlobal/,https://dev.azure.com/,Manufacture Quality Products
4,496004,Customer and Brand,Portfolio Epic,Implement,"Boyle, Paola Erin",https://dev.azure.com/BridgestoneGlobal/_apis/...,BoyleErin1@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NTA4ODUxNzAtZjg5Yy03ZWU2LTljYTYtOWJjNzgyMz...,2024-09-24T18:37:22.933Z,...,Support Marketing teams implementing B2C C360 ...,Related 2025 IT Goals: Fundamentally improve o...,,,2025.Q2; 2025.Q3; 2025.Q4; Enterprise & Custom...,,,BridgestoneGlobal/,https://dev.azure.com/,Customer and Brand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,586073,Secure and Protect the Enterprise,Portfolio Epic,Implement,Todd Thiel,https://dev.azure.com/BridgestoneGlobal/_apis/...,ThielTodd@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NDg2OWNmNWYtZjM3ZC03NWFjLWEzZGYtODQzOWUzMT...,2025-07-22T19:51:17.163Z,...,,,,,,,,BridgestoneGlobal/,https://dev.azure.com/,Secure and Protect the Enterprise
112,586075,Secure and Protect the Enterprise,Portfolio Epic,Implement,Todd Thiel,https://dev.azure.com/BridgestoneGlobal/_apis/...,ThielTodd@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NDg2OWNmNWYtZjM3ZC03NWFjLWEzZGYtODQzOWUzMT...,2025-07-22T19:52:35.177Z,...,,,,,,,,BridgestoneGlobal/,https://dev.azure.com/,Secure and Protect the Enterprise
113,586081,Secure and Protect the Enterprise,Portfolio Epic,Implement,Jeremy Pade,https://dev.azure.com/BridgestoneGlobal/_apis/...,PadeJeremy@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NTQ4ZGRmMjAtYmM2MC03YzJkLTlhMDktNDY3NGFmOD...,2025-07-22T20:17:16.31Z,...,,,,,,,,BridgestoneGlobal/,https://dev.azure.com/,Secure and Protect the Enterprise
114,586082,Secure and Protect the Enterprise,Portfolio Epic,Implement,Jeremy Pade,https://dev.azure.com/BridgestoneGlobal/_apis/...,PadeJeremy@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NTQ4ZGRmMjAtYmM2MC03YzJkLTlhMDktNDY3NGFmOD...,2025

In [31]:
def replace_spaces_with_percent20(x):
    # Replace every space with %20
    return x.replace(" ", "%20")

# Apply the function to the "TeamProjectUrl" column
IT_BigRocks_df["TeamProjectUrl"] = IT_BigRocks_df["TeamProjectUrl"].apply(replace_spaces_with_percent20)


In [32]:
IT_BigRocks_df.insert(len(IT_BigRocks_df.columns), "WorkItemEditUrl","/_workitems/edit/")

In [33]:
# Resetting index to ensure no duplication issues
IT_BigRocks_df = IT_BigRocks_df.reset_index(drop=True)  # Drop=True to avoid keeping the old index as a column

# Checking for duplicate columns (output will give a boolean series, which can be used to check for any duplicity issues)
duplicate_columns = IT_BigRocks_df.columns.duplicated()
duplicate_columns


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [34]:
IT_BigRocks_df = IT_BigRocks_df.reset_index(drop=True)
IT_BigRocks_df['WorkItemUrl'] = IT_BigRocks_df['AzureDevRootUrl'].astype(str) + IT_BigRocks_df['Organization'].astype(str) + IT_BigRocks_df['TeamProjectUrl'].astype(str) + IT_BigRocks_df['WorkItemEditUrl'] + IT_BigRocks_df['Id'].astype(str)

In [35]:
IT_BigRocks_df

,Id,TeamProject,WorkItemType,State,displayName,_links.avatar.href,uniqueName,imageUrl,descriptor,CreatedDate,...,LeadingIndicators,NonFunctionalRequirements,Tags,EstimatedStartDate,ValueDriver,Organization,AzureDevRootUrl,TeamProjectUrl,WorkItemEditUrl,WorkItemUrl
0,373586,Modern IT Management,Portfolio Epic,Implement,"Gade, Brent",https://dev.azure.com/BridgestoneGlobal/_apis/...,GadeBrent@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NT...,2023-08-07T15:53:28.51Z,...,Mainframe migration Q4 2024 Migration Waves 13...,,,,,BridgestoneGlobal/,https://dev.azure.com/,Modern%20IT%20Management,/_workitems/edit/,https://dev.azure.com/BridgestoneGlobal/Modern...
1,373588,Modern IT Management,Portfolio Epic,Implement,"Gade, Brent",https://dev.azure.com/BridgestoneGlobal/_apis/...,GadeBrent@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.MzNlYmQ0NWMtMThhYy03NzYwLThlZDctOGU1Zjk4NT...,2023-08-07T15:53:47.827Z,...,,,,,,BridgestoneGlobal/,https://dev.azure.com/,Modern%20IT%20Management,/_workitems/edit/,https://dev.azure.com/BridgestoneGlobal/Modern...
2,441625,Manufacture Quality Products,Portfolio Epic,Plan,"Harden, Wendy",https://dev.azure.com/BridgestoneGlobal/_apis/...,HardenWendy@BFUSA.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.YjMzZDRjMTItNjQwZS03NjhmLWJlYjgtMjE4Zjk0YT...,2024-03-29T06:26:35.01Z,...,,,Big Rock,,Blue Dolars,BridgestoneGlobal/,https://dev.azure.com/,Manufacture%20Quality%20Products,/_workitems/edit/,https://dev.azure.com/BridgestoneGlobal/Manufa...
3,475391,Manufacture Quality Products,Portfolio Epic,Plan,"Harden, Wendy",https://dev.azure.com/BridgestoneGlobal/_apis/...,HardenWendy@BFUSA.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.YjMzZDRjMTItNjQwZS03NjhmLWJlYjgtMjE4Zjk0YT...,2024-07-12T17:53:48.32Z,...,,,Big Rock,,Blue Dolars,BridgestoneGlobal/,https://dev.azure.com/,Manufacture%20Quality%20Products,/_workitems/edit/,https://dev.azure.com/BridgestoneGlobal/Manufa...
4,496004,Customer and Brand,Portfolio Epic,Implement,"Boyle, Paola Erin",https://dev.azure.com/BridgestoneGlobal/_apis/...,BoyleErin1@bfusa.com,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NTA4ODUxNzAtZjg5Yy03ZWU2LTljYTYtOWJjNzgyMz...,2024-09-24T18:37:22.933Z,...,,,2025.Q2; 2025.Q3; 2025.Q4; Enterprise & Custom...,,,BridgestoneGlobal/,https://dev.azure.com/,Customer%20and%20Brand,/_workitems/edit/,https://dev.azure.com/BridgestoneGlobal/Custom...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,586073,Secure and Protect the Enterprise,Portfolio Epic,Implement,Todd Thiel,https://dev.azure.com/BridgestoneGlobal/_apis/...,ThielTodd@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NDg2OWNmNWYtZjM3ZC03NWFjLWEzZGYtODQzOWUzMT...,2025-07-22T19:51:17.163Z,...,,,,,,BridgestoneGlobal/,https://dev.azure.com/,Secure%20and%20Protect%20the%20Enterprise,/_workitems/edit/,https://dev.azure.com/BridgestoneGlobal/Secure...
112,586075,Secure and Protect the Enterprise,Portfolio Epic,Implement,Todd Thiel,https://dev.azure.com/BridgestoneGlobal/_apis/...,ThielTodd@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NDg2OWNmNWYtZjM3ZC03NWFjLWEzZGYtODQzOWUzMT...,2025-07-22T19:52:35.177Z,...,,,,,,BridgestoneGlobal/,https://dev.azure.com/,Secure%20and%20Protect%20the%20Enterprise,/_workitems/edit/,https://dev.azure.com/BridgestoneGlobal/Secure...
113,586081,Secure and Protect the Enterprise,Portfolio Epic,Implement,Jeremy Pade,https://dev.azure.com/BridgestoneGlobal/_apis/...,PadeJeremy@BFUSA.COM,https://dev.azure.com/BridgestoneGlobal/_apis/...,aad.NTQ4ZGRmMjAtYmM2MC03YzJkLTlhMDktNDY3NGFmOD...,2025-07-22T20:17:16.31Z,...,,,,,,BridgestoneGlobal/,https://dev.azure.com/,Secure%20and%20Protect%20the%20Enterprise,/_workitems/edit/,https://dev.azure.com/BridgestoneGlobal/Secure...
114,586082,Secure and Protect the Enterprise,Portfolio Epic,Implement,Jeremy Pade,https://dev.azure.com/BridgestoneGlobal/_apis/.